# Estimating a Yield Curve Model

## Loading Packages

In [1]:
using Optim

include("jlFiles/printmat.jl")
include("jlFiles/lagnPs.jl")

lagnPs (generic function with 2 methods)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
    default(show=false)               #for pyplot: avoids pop-ups
else    
    gr(size=(600,400))
    default(show=true)
end

In [3]:
xx = readdlm("Data/USCMRatesPs.csv",',',header=true)
y  = xx[1]

YearMonth = 1970 + 1/24 + (1/12)*collect(0:size(y,1)-1)

y[:,1] = ((1-0.25*y[:,1]/100).^(-1/0.25) - 1)*100    #discount basis->effective
y[:,2] = ((1-0.50*y[:,2]/100).^(-1/0.5) - 1)*100

m      = [0.25 0.5 1 3 5 7 10]                             #time to maturity (in years)
mMonth = broadcast(x->round(Int,x),m*12)                   #maturites, in months (integers)
y      = log.(1+y/100)                    #continuously compounded interest rates
y      = y/12                            #interest rates per month (the period length of data)

(T,n) = size(y)

(520,7)

## Establishing Notation for an Affine Yield Curve Model


An affine yield curve model implies that the yield on an $n$-period discount
bond is

$$
\begin{equation}
y_{nt}=a_{n}+b_{n}^{\prime}x_{t}\nonumber
\end{equation}
$$

where $x_{t}$ is an $K\times1$ vector of state variables. Here $a_{n}$ is a
scalar and $b_{n}$ is a $K\times1$ vector (so $b_{n}^{\prime}x_{t}$ is a scalar). 

Transpose to get
$$
y_{nt}=a_{n}+\underset{1\times K}{\underbrace{x_{t}^{\prime}}}
\underset{K\times1}{\underbrace{b_{n}}}
$$

As an example, consider two different maturities ($n$), denoted $1$ and $2$.
The row vector is then

$$
\begin{bmatrix}
y_{1t} & y_{2t}
\end{bmatrix} =
\begin{bmatrix}
a_{1} & a_{2}
\end{bmatrix}
+x_{t}^{\prime}
\begin{bmatrix}
b_{1} & b_{2}
\end{bmatrix}.
$$

Now, stack observations ($t=1$ to $T$) vertically

$$
Y_{T\times n}=\mathbf{1}_{T\times1}\otimes a_{1\times n}+X_{T\times K}b_{K\times n},
$$

where $\mathbf{1}_{T\times1}$ is a $T\times1$ vector of ones and $\otimes$ is
the Kronecker product.

An example of the Kronecker product:
$$
\mathbf{1}_{2\times1}\otimes
\begin{bmatrix}
1 & 2 & 3
\end{bmatrix} =
\begin{bmatrix}
1 & 2 & 3\\
1 & 2 & 3
\end{bmatrix}.
$$

### From Model Parameters to $a_{n}$ and $b_{n}$

The following function VasicekABFn.jl takes the
Vasicek model parameters as inputs and calculates the $a_{n}$ and $b_{n}$ values.

In [4]:
function VasicekABFn(lambda,mu,rho,sigma,nMo,nMu,yo)

  nMax = maximum([nMo;nMu])    #longest maturity to calculate (a,b) for
  Nvec = 1:nMax

  A = fill(NaN,(1,nMax))              #recursive solution of AR(1) model
  B = fill(NaN,(1,nMax))
  B[1] = 1 + 0*rho
  A[1] = 0 + 0*(1-rho)*mu - (lambda+0)^2*sigma^2/2
  for n = 2:nMax
    B[n] = 1 + B[n-1]*rho
    A[n] = A[n-1] + B[n-1]*(1-rho)*mu - (lambda+B[n-1])^2*sigma^2/2
  end

  a  = A./Nvec'
  b  = B./Nvec'

  ao = a[:,nMo][1]                       #required maturities
  bo = b[:,nMo][1]                       #[1] to make sure the are scalar

  xt    = (yo-ao)/bo                     #value of state variable x(t)

  au    = a[:,nMu]
  bu    = b[:,nMu]
  yuHat = au .+ xt*bu                   #Txn, fitted values of yu

  return ao,bo,xt,au,bu,yuHat

end

VasicekABFn (generic function with 1 method)

### Testing the VasicekABFn Function

In [5]:
vvo = 1                               #indices of yields that are observed without errors
vvu = 2:n                             #indices of yields with observation errors

yo = y[:,vvo]                         #observable yields
yu = y[:,vvu]                         #unobservable yields
nMo = mMonth[vvo]                     #maturity of yo
nMu = mMonth[vvu]                     #maturity of yu


#just testing the Vasicek functions
(ao,bo,xt,au,bu,yuHat) = VasicekABFn(1,0.5,0.9,0.02,nMo,nMu,yo)

println("ao and bo: ")
printmat([ao bo])

plot(YearMonth,xt,legend=false,xlims=(1970,2014))
title!("xt (implied by ao and bo, using specific parameter values)")

ao and bo: 
     0.047     0.903



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1970 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -0.050 
 
 
 -0.045 
 
 
 -0.040 
 
 
 xt (implied by ao and bo, using specific parameter values) 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 57.7806,212.368 58.801,228.763 59.8214,239.824 60.8419,242.476 61.8623,235.18 62.8828,238.718 63.9032,243.802 64.9236,244.686 65.9441,251.091 66.9645,255.726 
 67.985,269.617 69.0054,278.645 70.0258,288.103 71.0463,304.356 72.0667,311.375 73.0872,300.845 74.1076,294.696 75.1281,281.285 76.1485,266.973 77.1689,277.104 
 78.1894,282.605 79.2098,287.663 80.2303,292.938 81.2507,297.551 82.2711,311.375 83.2916,315.321 84.312,303.698 85.3325,304.137 86.3529,304.576 87.3733,299.747 
 88.3938,298.21 89.4142,297.332 90.4347,283.265 91.4551,281.505 92.4755,280.625 93.496,274.242 94.5164,266.752 95.5369,262.564 96.5573,251.753 97.5777,247.999 
 98.5982,245.791 99.6186,227.435 100.639,209.263 101.66,194.609 102.68,203.049 103.7,226.771 104.721,213.255 105.741,221.677 106.762,214.586 107.782,228.985 
 108.803,210.372 109.823,202.161 110.843,204.381 111.864,211.703 112.884,219.462 113.905,188.162 114.925,208.154 115.946,221.456 116.966,221.234 117.987,228.321 
 119.007,247.999 120.027,264.769 121.048,264.989 122.068,262.343 123.089,270.718 124.109,268.295 125.13,250.87 126.15,244.023 127.171,244.465 128.191,254.623 
 129.211,265.209 130.232,266.091 131.252,278.645 132.273,278.425 133.293,275.783 134.314,278.865 135.334,271.379 136.354,266.752 137.375,270.718 138.395,272.7 
 139.416,274.022 140.436,277.544 141.457,281.285 142.477,290.081 143.498,284.145 144.518,283.045 145.538,284.585 146.559,285.904 147.579,276.664 148.6,275.343 
 149.62,271.599 150.641,264.989 151.661,257.932 152.682,250.208 153.702,251.532 154.722,252.195 155.743,244.023 156.763,243.802 157.784,247.337 158.804,247.337 
 159.825,244.686 160.845,237.613 161.866,231.419 162.886,229.87 163.906,212.812 164.927,209.707 165.947,195.276 166.968,185.493 167.988,179.485 169.009,180.153 
 170.029,176.591 171.049,177.036 172.07,173.695 173.09,185.938 174.111,181.933 175.131,175.7 176.152,159.204 177.172,127.015 178.193,124.999 179.213,119.397 
 180.233,120.294 181.254,100.997 182.274,48.2759 183.295,93.3568 184.315,196.609 185.336,230.091 186.356,208.154 187.377,184.381 188.397,158.981 189.417,128.806 
 190.438,81.4331 191.458,41.7197 192.479,52.3428 193.499,57.5366 194.52,89.7589 195.54,82.3336 196.56,23.3815 197.581,58.8909 198.601,53.9238 199.622,41.2674 
 200.642,59.5681 201.663,85.7095 202.683,145.806 203.704,146.03 204.724,114.016 205.744,87.0595 206.765,105.04 207.785,104.591 208.806,118.276 209.826,109.753 
 210.847,134.85 211.867,194.387 212.888,211.259 213.908,215.916 214.928,207.932 215.949,210.816 216.969,212.59 217.99,207.044 219.01,201.717 220.031,204.825 
 221.051,205.269 222.072,191.942 223.092,185.493 224.112,179.708 225.133,187.273 226.153,195.276 227.174,192.609 228.194,187.273 229.215,189.496 230.235,185.271 
 231.255,175.7 232.276,171.913 233.296,168.793 234.317,167.902 235.337,162.327 236.358,154.517 237.378,156.75 238.399,170.799 239.419,195.942 240.439,208.154 
 241.46,214.807 242.48,203.493 243.501,197.942 244.521,210.594 245.542,221.013 246.562,232.747 247.583,229.87 248.603,228.542 249.623,229.428 250.644,228.099 
 251.664,226.328 252.685,229.428 253.705,230.091 254.726,230.313 255.746,241.371 256.766,252.415 257.787,250.429 258.807,249.104 259.828,257.491 260.848,264.107 
 261.869,271.159 262.889,271.82 263.91,268.074 264.93,264.107 265.95,266.311 266.971,262.784 267.991,262.784 269.012,261.682 270.032,261.241 271.053,261.02 
 272.073,260.579 273.094,252.857 274.114,244.907 275.134,250.87 276.155,260.579 277.175,258.815 278.196,257.932 279.216,261.241 280.237,260.359 281.257,255.726 
 282.277,247.999 283.298,243.581 284.318,237.613 285.339,230.313 286.359,226.328 287.38,223.892 

## The Likelihood Function

To formulate the log-liklihood, the VasicekTsCsFn
first takes the parameter vectors and transforms the parameters (scaling,
making sure that $-1\leq\rho\leq1$, etc) and the calls on the function
VasicekABFn (see above).


From the previous output, the code in VasicekTsCsFn moves on to
define the 1-step ahead forecast errors of $y_{ot}$ as

$$
v_{t}=y_{ot}-E_{t-1}y_{ot}
$$

and its variance $S=b_{o}^{\prime}\sigma^{2}b_{o}$. The contribution to the
likelihood function in $t$ is proportional to $v_{t}^{\prime}S^{-1}v_{t}$.
Then, the code defines the cross-sectional errors (for $y_{ut}$) as

$$
u_{t}=y_{ut}-\hat{y}_{ut} \ \text{, where } \ \hat{y}_{ut}=a_{u}+b_{u}^{\prime}x_{t},
$$

The contribution to the likelihood function is $u_{t}^{\prime}\Omega^{-1}
u_{t}$, where the code assumes that $\Omega$ is diagonal,
$
\Omega=I\omega^{2},
$
where $\omega$ is one of the parameters estimated. 

The function VasicekTsCsLossFn creates
the value of the function that will be minimized.

In [6]:
function VasicekTsCsFn(par,yo,yu,nMo,nMu)
#VasicekTsCsFn    Loss function for estimating the Vasicek model using both
#                 time series and cross sectional information

  J = length(nMu)
  T = size(yo,1)

  lambda  = par[1]*100
  mu      = par[2]/1200
  p       = 1 - 2/(1+exp(par[3]))          #inverse: par[3] = log((1+p)/(1-p))
  s2      = (par[4]/1200)^2
  omega_i = abs(par[5]/1200)

  if length(nMo) != 1
    error("yo must be a single yield")
  end

  (ao,bo,xt,au,bu,yuHat) = VasicekABFn(lambda,mu,p,sqrt(s2),nMo,nMu,yo)

  Et1xt    = (1-p)*mu + p*lagnPs(xt)    #E(t-1)x(t)
  Et1xt[1] = mu
  Et1yo    = ao + Et1xt*bo              #E(t-1)yo(t)
  v        = yo - Et1yo                 #Tx1, forecast error of yo
  S        = bo's2*bo                   #variance of forecast error of yo
  S_1      = inv(S*1000)*1000           #1000/1000 improves the precision a bit
  LLo_t    = -0.5*log(2*pi) - 0.5*log(det(S)) - 0.5*sum(v*S_1.*v,2)   #Tx1

  u        = yu -  yuHat                 #TxL, fitted errors of yu
  Omega    = eye(J)*omega_i^2
  Omega_1  = inv(Omega)                  #covariance matrix of u
  LLu_t    = -0.5*J*log(2*pi) - 0.5*log(det(Omega)) - 0.5*sum(u*Omega_1.*u,2)  #Tx1

  LL_t    = LLo_t + LLu_t                #Tx1, log likelihood(t), sum of TS and CS
  
  return LL_t,yuHat,u,xt

end


function VasicekTsCsLossFn(par,yo,yu,nMo,nMu)
  LL_t, = VasicekTsCsFn(par,yo,yu,nMo,nMu)
  MinusLL = -sum(LL_t[2:end])            #to be minimized 
  return MinusLL
end

VasicekTsCsLossFn (generic function with 1 method)

In [7]:
#notice: important that the elements of the parameter vector have similar magnitude
#lambda/100,mu*1200,p = 1 - 2/(1+exp(par(3))) so 5.6 -> p = 0.993,sigma*1200,omega*1200
par0 = [-2.3;10;5.5;0.5;0.8]

MinusLL = VasicekTsCsLossFn(par0,yo,yu,nMo,nMu)

printlnPs("The negative of the log-likelihood function at the starting guess: ",MinusLL)

The negative of the log-likelihood function at the starting guess: 

In [8]:
                             #do MLE by optimization with optimize, minimize -sum(LL)
Sol = optimize(par->VasicekTsCsLossFn(par,yo,yu,nMo,nMu),par0)
par1 = Optim.minimizer(Sol)

println("\npar0 and par1")
printmat([par0 par1])

#converting back to model parameters 
(lambda,mu,rho,s2) = (par1[1]*100,par1[2]/1200,1 - 2/(1+exp(par1[3])),par1[4]/1200)
println("Estimated (lambda,mu,rho,s2):")
printmat([lambda mu rho s2])


par0 and par1
    -2.300    -2.441
    10.000     9.672
     5.500     5.753
     0.500     0.525
     0.800     0.797

Estimated (lambda,mu,rho,s2):
  -244.095     0.008     0.994     0.000



In [9]:
                                               #fitted yields at parameter estimates
MinusLL = VasicekTsCsLossFn(par1,yo,yu,nMo,nMu)
println("The negative of the log-likelihood function at the point estimate: ",MinusLL)

(LL_t,yuHat,u,xt) = VasicekTsCsFn(par1,yo,yu,nMo,nMu)

yhatTsCs = [yo yuHat]                      #also yo

                                            #a and b at estimated parameters, average yield curve
(ao,bo,xt,au,bu,yuHatAvg) = VasicekABFn(lambda,mu,rho,s2,nMo,nMu,mean(yo))
yuHatAvg = [mean(yo) yuHatAvg]             #also yo



yy = y[:,[3,6]] - yhatTsCs[:,[3,6]]
  
plot(YearMonth,12*yy[:,1],label="1 year")
plot!(YearMonth,12*yy[:,2],line=(:dot,2),label="7 years")
plot!(xlims= (1970,2014),ylims=(-0.04,0.04))
title!("Pricing errors, Vasicek (TS and CS)")

The negative of the log-likelihood function at the point estimate: -21671.211618944784


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1970 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -0.04 
 
 
 -0.02 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 Pricing errors, Vasicek (TS and CS) 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 48.9001,215.914 49.9374,204.155 50.9746,207.924 52.0119,198.763 53.0492,184.471 54.0865,185.718 55.1237,194.395 56.161,197.676 57.1983,195.306 58.2356,198.661 
 59.2728,209.828 60.3101,213.483 61.3474,212.916 62.3847,209.557 63.4219,204.081 64.4592,198.852 65.4965,179.324 66.5338,180.682 67.5711,192.525 68.6083,182.292 
 69.6456,187.865 70.6829,199.133 71.7202,198.824 72.7574,192.526 73.7947,178.437 74.832,170.897 75.8693,177.081 76.9065,163.689 77.9438,176.602 78.9811,173.852 
 80.0184,175.666 81.0556,176.58 82.0929,181.818 83.1302,185.374 84.1675,197.874 85.2047,200.053 86.242,199.368 87.2793,195.046 88.3166,188.889 89.3538,196.475 
 90.3911,199.247 91.4284,218.617 92.4657,210.081 93.5029,221.8 94.5402,225.975 95.5775,216.17 96.6148,236.335 97.652,231.938 98.6893,239.954 99.7266,233.632 
 100.764,234.186 101.801,214.853 102.838,203.714 103.876,193.49 104.913,172.407 105.95,212.456 106.988,192.352 108.025,199.64 109.062,216.431 110.099,216.827 
 111.137,197.321 112.174,199.537 113.211,193.551 114.248,165.373 115.286,170.063 116.323,179.21 117.36,179.689 118.398,168.699 119.435,165.708 120.472,178.862 
 121.509,176.949 122.547,170.501 123.584,178.751 124.621,174.926 125.658,167.48 126.696,173.61 127.733,168.303 128.77,172.558 129.808,178.141 130.845,182.655 
 131.882,186.812 132.919,194.236 133.957,195.681 134.994,195.108 136.031,189.902 137.069,184.404 138.106,180.008 139.143,179.912 140.18,181.473 141.218,185.851 
 142.255,187.439 143.292,182.49 144.329,189.958 145.367,186.939 146.404,185.107 147.441,183.346 148.479,186.359 149.516,184.278 150.553,178.398 151.59,172.506 
 152.628,162.326 153.665,165.316 154.702,165.306 155.739,171.774 156.777,192.497 157.814,178.019 158.851,171.297 159.889,179.167 160.926,186.804 161.963,192.419 
 163,199.208 164.038,203.64 165.075,210.391 166.112,208.28 167.15,213.487 168.187,212.09 169.224,210.221 170.261,210.497 171.299,216.581 172.336,244.419 
 173.373,239.38 174.41,203.973 175.448,235.758 176.485,244.078 177.522,194.147 178.56,177.59 179.597,201.494 180.634,183.875 181.671,183.029 182.709,204.867 
 183.746,234.394 184.783,285.838 185.82,295.954 186.858,266.087 187.895,235.023 188.932,225.848 189.97,271.262 191.007,251.924 192.044,228.248 193.081,214.936 
 194.119,185.704 195.156,177.312 196.193,173.727 197.231,155.619 198.268,161.822 199.305,200.117 200.342,194.614 201.38,194.331 202.417,192.124 203.454,180.331 
 204.491,162.621 205.529,115.128 206.566,104.617 207.603,158.023 208.641,180.777 209.678,185.312 210.715,193.777 211.752,192.519 212.79,198.308 213.827,194.515 
 214.864,196.937 215.901,192.436 216.939,183.267 217.976,181.445 219.013,181.312 220.051,179.521 221.088,179.563 222.125,183.364 223.162,187.498 224.2,190.283 
 225.237,187.091 226.274,182.092 227.312,157.53 228.349,142.363 229.386,155.645 230.423,179.908 231.461,185.108 232.498,184.344 233.535,177.763 234.572,173.296 
 235.61,172.683 236.647,184.368 237.684,172.076 238.722,176.226 239.759,183.418 240.796,187.292 241.833,190.591 242.871,185.32 243.908,182.694 244.945,187.888 
 245.982,196.91 247.02,199.454 248.057,195.594 249.094,200.184 250.132,202.262 251.169,204.927 252.206,200.034 253.243,199.322 254.281,201.893 255.318,203.007 
 256.355,195.588 257.392,196.39 258.43,200.539 259.467,205.567 260.504,204.957 261.542,204.4 262.579,201.415 263.616,183.653 264.653,163.375 265.691,172.277 
 266.728,178.249 267.765,179.051 268.803,168.172 269.84,159.48 270.877,166.402 271.914,161.103 272.952,170.484 273.989,178.607 275.026,177.424 276.063,174.098 
 277.101,173.27 278.138,178.413 279.175,179.555 280.213,176.725 281.25,188.121 282.287,192.207 28

In [10]:
plot(m',12*yuHatAvg',label="at average x[t] ")
plot!(m',12*mean(y,1)',line=(:dot,2),label="data")

plot!(xlims=(0,11),ylims=(0.05,0.08))
title!("Avg yield curve, Vasicek (TS and CS)")
xlabel!("Maturity (years)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.050 
 
 
 0.055 
 
 
 0.060 
 
 
 0.065 
 
 
 0.070 
 
 
 0.075 
 
 
 0.080 
 
 
 Avg yield curve, Vasicek (TS and CS) 
 
 
 Maturity (years) 
 
 
 
 
 
 
 
 at average x[t] 
 
 
 
 data

In [11]:
bOls = [ones(T) y[:,1]]\y                 #LS of yields on short yield
alfa = bOls[1:1,:]
beta = bOls[2:2,:]
                         #rescaling Vasicek au and bu to be comaparable with OLS on yo
bu_b = bu/bo             #y = au + bu*xt, but xt = (yo-ao)/bo
au_b = au - bu/bo*ao     #y = au + bu*(yo-ao)/bo = y = au - bu*ao/bo + bu/bo*yo
bu_b = [1 bu_b]          #also yo
au_b = [0 au_b]


plot(m',alfa'*1200,label="Vasicek")
plot!(m',au_b'*1200,line=(:dot,2),label="OLS")
plot!(xlims=(0,maximum(m)+1))
title!("Intercept times 1200, Vasicek and OLS")
xlabel!("maturity (years)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 Intercept times 1200, Vasicek and OLS 
 
 
 maturity (years) 
 
 
 
 
 
 
 
 Vasicek 
 
 
 
 OLS

In [12]:
plot(m',beta',label="Vasicek")
plot!(m',bu_b',line=(:dot,2),label="OLS")
plot!(xlims=(0,maximum(m)+1))
title!("Slope on short rate, Vasicek and OLS")
xlabel!("maturity (years)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 Slope on short rate, Vasicek and OLS 
 
 
 maturity (years) 
 
 
 
 
 
 
 
 Vasicek 
 
 
 
 OLS